In [140]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import time
import datetime

In [141]:
df = pd.read_csv('https://raw.githubusercontent.com/daniel-dc-cd/data_science/master/module_4_ML/data/seattle_weather_1948-2017.csv')

numrows = 25549 



In [142]:
#create an empty dataframe to hold 100 values
heuristic_df = pd.DataFrame({'yesterday':[0.0]*numrows,
                             'today':[0.0]*numrows,
                             'tomorrow':[0.0]*numrows,
                             'guess':[False]*numrows,  # logical guess
                             'rain_tomorrow':[False]*numrows,  # historical observation
                             'correct':[False]*numrows,  # TRUE if your guess matches the historical observation
                             'true_positive':[False]*numrows,  # TRUE If you said it would rain and it did
                             'false_positive':[False]*numrows,  # TRUE If you sait id would rain and it didn't
                             'true_negative':[False]*numrows,  # TRUE if you said it wouldn't rain and it didn't
                             'false_negative':[False]*numrows})  # TRUE if you said it wouldn't raing and it did

# sort columns for convience
seq = ['yesterday',
       'today',
       'tomorrow',
       'guess',
       'rain_tomorrow',
       'correct',
       'true_positive',
       'false_positive',
       'true_negative',
       'false_negative']

heuristic_df = heuristic_df.reindex(columns=seq)

In [143]:
df#.head()

,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,0.47,51,42,True
1,1948-01-02,0.59,45,36,True
2,1948-01-03,0.42,45,35,True
3,1948-01-04,0.31,45,34,True
4,1948-01-05,0.17,45,32,True
...,...,...,...,...,...
25546,2017-12-10,0.00,49,34,False
25547,2017-12-11,0.00,49,29,False
25548,2017-12-12,0.00,46,32,False
25549,2017-12-13,0.00,48,34,False


In [144]:
heuristic_df#.head()

,yesterday,today,tomorrow,guess,rain_tomorrow,correct,true_positive,false_positive,true_negative,false_negative
0,0.0,0.0,0.0,False,False,False,False,False,False,False
1,0.0,0.0,0.0,False,False,False,False,False,False,False
2,0.0,0.0,0.0,False,False,False,False,False,False,False
3,0.0,0.0,0.0,False,False,False,False,False,False,False
4,0.0,0.0,0.0,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
25544,0.0,0.0,0.0,False,False,False,False,False,False,False
25545,0.0,0.0,0.0,False,False,False,False,False,False,False
25546,0.0,0.0,0.0,False,False,False,False,False,False,False
25547,0.0,0.0,0.0,False,False,False,False,False,False,False


In [145]:
for z in range(numrows):
    #start at time 2 in the data frame
    i = z + 2
    #pull values from the dataframe
    yesterday = df.iloc[(i-2),1]
    today = df.iloc[(i-1),1]
    tomorrow = df.iloc[i,1]
    rain_tomorrow = df.iloc[(i),1]
    
    heuristic_df.iat[z,0] = yesterday
    heuristic_df.iat[z,1] = today
    heuristic_df.iat[z,2] = tomorrow
    heuristic_df.iat[z,3] = False # set guess default to False
    heuristic_df.iat[z,4] = rain_tomorrow
    
    #example hueristic
    if today > 0.0 and yesterday > 0.0:
        heuristic_df.iat[z,3] = True
        
    if heuristic_df.iat[z,3] == heuristic_df.iat[z,4]:
        heuristic_df.iat[z,5] = True
        if heuristic_df.iat[z,3] == True:
            heuristic_df.iat[z,6] = True #true positive
        else:
            heuristic_df.iat[z,8] = True #true negative
    else:
        heuristic_df.iat[z,5] = False
        if heuristic_df.iat[z,3] == True:
            heuristic_df.iat[z,7] = True #false positive
        else:
            heuristic_df.iat[z,9] = True #false negative

In [146]:
heuristic_df

,yesterday,today,tomorrow,guess,rain_tomorrow,correct,true_positive,false_positive,true_negative,false_negative
0,0.47,0.59,0.42,True,True,True,True,False,False,False
1,0.59,0.42,0.31,True,True,True,True,False,False,False
2,0.42,0.31,0.17,True,True,True,True,False,False,False
3,0.31,0.17,0.44,True,True,True,True,False,False,False
4,0.17,0.44,0.41,True,True,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...
25544,0.00,0.00,0.00,False,False,True,False,False,True,False
25545,0.00,0.00,0.00,False,False,True,False,False,True,False
25546,0.00,0.00,0.00,False,False,True,False,False,True,False
25547,0.00,0.00,0.00,False,False,True,False,False,True,False


In [147]:
from sklearn.model_selection import train_test_split 

# enter split function here to make h_train and h_test subsets of the data
X_train, X_test, y_train, y_test = train_test_split(df, df['RAIN'], test_size=0.3)

X_train

,DATE,PRCP,TMAX,TMIN,RAIN
13029,1983-09-03,0.00,66,58,False
16003,1991-10-25,0.15,47,42,True
12356,1981-10-30,0.11,58,43,True
284,1948-10-11,0.00,56,43,False
20147,2003-02-28,0.07,45,40,True
...,...,...,...,...,...
2836,1955-10-07,0.67,51,45,True
9675,1974-06-28,0.00,75,51,False
9541,1974-02-14,0.09,51,43,True
19604,2001-09-03,0.03,68,55,True


In [148]:
X_test

,DATE,PRCP,TMAX,TMIN,RAIN
15532,1990-07-11,0.00,89,62,False
7649,1968-12-10,0.72,45,41,True
20216,2003-05-08,0.00,60,42,False
12190,1981-05-17,0.02,60,46,True
14868,1988-09-15,0.00,61,50,False
...,...,...,...,...,...
3281,1956-12-25,0.00,49,29,False
9948,1975-03-28,0.00,54,31,False
9224,1973-04-03,0.00,61,37,False
10866,1977-10-01,0.00,61,50,False


In [149]:
# we used this simple approach in the first part to see what percent of the time we where correct 
# calculated as (true positive + true negative)/ number of guesses
heuristic_df['correct'].value_counts()/numrows

True     0.671611
False    0.328389
Name: correct, dtype: float64

In [150]:
# precision is the percent of your postive prediction which are correct
# more specifically it is calculated (num true positive)/(num tru positive + num false positive)

heuristic_df['true_positive'].sum() / (heuristic_df['true_positive'].sum() + heuristic_df['false_positive'].sum())

0.674109000138677

In [151]:
# recall the percent of the time you are correct when you predict positive
# more specifically it is calculated (num true positive)/(num tru positive + num false negative)

heuristic_df['true_positive'].sum() / (heuristic_df['true_positive'].sum() + heuristic_df['false_negative'].sum())

0.44592239244106047

In [152]:
#SSE


import numpy as np

def sse(y_true, y_pred):
    '''returns sum of squared errors (actual vs model)'''
    squared_errors = (y_true - y_pred) ** 2
    return np.sum(squared_errors)


sse(y_true=heuristic_df['rain_tomorrow'].astype('int'), y_pred=heuristic_df['guess'].astype('int'))

8390

In [153]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_true=heuristic_df['rain_tomorrow'].astype('int'), y_pred=heuristic_df['guess'].astype('int'))

0.3283885866374418